# Lab session 1: Comparison of supervised learning models for spam classification

## Task specification
In this lab session, you will follow the demo code provided below, and complete the following two tasks:
* Compare and analyze the performance of several popular **supervised learning** models on a number of simulated classification tasks. 
* Apply the model of your choice to a **spam filtering** task, and report the performance of your model.

## Learning objectives
By the end of this lab, you should understand the following:
* How to *apply machine learning models* in python such as multi-layer perceptron, logistic regression and Gaussian Naive Bayes?
* How do different classification algorithms behave on *different data distributions*?
* What are the *design choices* of classification models? When should we care about *training efficiency*, *label complexity*, and *accuracy*?
* How to *evaluate and visualize* the outcome of classification models?

In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

from scipy import stats
from scipy import linalg

from numpy.random import RandomState

# Machine Learning library. 

# General math and plotting modules.
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.special import erfinv

# Widget and formatting modules
import IPython
import ipywidgets
from ipywidgets import interact, interactive, interact_manual, fixed
from matplotlib import rcParams
# If in your browser the figures are not nicely vizualized, change the following line. 
rcParams['figure.figsize'] = (10, 5)
rcParams['font.size'] = 16

# Machine Learning library. 
from sklearn import datasets
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


## Problem setup: synthetic dataset

To understand the behavior of different supervised models, we would like to have full control over the training dataset. Now let's create a few interesting synthetic datasets that capture specific types of data distributions.


In [2]:
def get_classification_dataset(dataset, n_samples=200, noise=0.3):
    if dataset == 'linear':
        X, Y = linear_separable_data(n_samples, noise=noise, dim=2) 
        Y = (Y + 1) // 2
    elif dataset == '2-blobs':
        X, Y = datasets.make_classification(n_classes=2, n_features=2, n_informative=2, n_redundant=0,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8)
    elif dataset == '3-blobs':
        X, Y = datasets.make_classification(n_classes=3, n_features=2, n_informative=2, n_redundant=0,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8)
    elif dataset == '4-blobs':
        X, Y = datasets.make_classification(n_classes=4, n_features=2, n_informative=2, n_redundant=0,
                                            n_clusters_per_class=1, n_samples=n_samples, random_state=8) 
    elif dataset == 'circles':
        X, Y = datasets.make_circles(n_samples=n_samples, factor=.5, noise=.05)
    elif dataset == 'moons':
        X, Y = datasets.make_moons(n_samples=n_samples, noise=.05)
    elif dataset == 'iris':
        X, Y = datasets.load_iris(return_X_y=True)
        X = X[:, :2]

    return X, Y


def linear_separable_data(num_positive, num_negative=None, noise=0., offset=1, dim=2):
    if num_negative is None:
        num_negative = num_positive

    x = offset + noise * np.random.randn(num_positive, dim)
    y = 1 * np.ones((num_positive,), dtype=np.int)

    x = np.concatenate((x, noise * np.random.randn(num_negative, dim)), axis=0)
    y = np.concatenate((y, -1 * np.ones((num_negative,), dtype=np.int)), axis=0)

    x = np.concatenate((x, np.ones((num_positive + num_negative, 1))), axis=1)

    return x, y


## Preliminaries: helper functions for data analytics and visualization

We also need several helper functions for visualizing the data distribution and qualitatively assessing the model behavior.

In [30]:
def plot_ellipse(mean, covar, color, ax):
    v, w = linalg.eigh(covar)
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    u = w[0] / linalg.norm(w[0])

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan(u[1] / u[0])
    angle = 180. * angle / np.pi  # convert to degrees


    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180. + angle,  color=color)
    ell.set_clip_box(ax.bbox)
    ell.set_alpha(0.5)
    ax.add_artist(ell)


def plot_model(model, X, Y, means, covariances):
    num_classes = len(np.unique(Y))
    cmap = plt.cm.jet
    pmap = plt.cm.cividis_r
    norm = mpl.colors.Normalize(vmin=0, vmax=num_classes - 1)
        
    # PREDICT
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = .02  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    xy = np.c_[xx.ravel(), yy.ravel()]
    
    # predicted class label
    C = model.predict(xy)
    # prediction probability
    P = model.predict_proba(xy)
    # uncertainty of the label captured by the predictive entropy
    H = -(P * model.predict_log_proba(xy)).sum(axis=1)
    
    P = P.max(axis=1)
    # Put the result into a color plot
    C = C.reshape(xx.shape)
    P = P.reshape(xx.shape)
    H = H.reshape(xx.shape)
    
    # PLOTS
    fig, axes = plt.subplots(1, 3)

    for ax in axes:
        ax.scatter(X[:, 0], X[:, 1], c=Y, cmap=plt.cm.jet)
        
        if means is not None:
            for i in range(num_classes):
                plot_ellipse(means[i], covariances[i], cmap(norm(i)), ax)

        ax.set_xlim(xx.min()+h, xx.max()-h)
        ax.set_ylim(yy.min()+h, yy.max()-h)
        ax.set_xticks(())
        ax.set_yticks(())
        ax.set_aspect('equal')

    axes[0].set_title('Classification Boundary')
    axes[0].contourf(xx, yy, C, cmap=cmap, alpha=0.5)
    
    axes[2].set_title('Prediction Probabilities')
    cf = axes[2].contourf(xx, yy, P, cmap=pmap, alpha=0.5, vmin=1. / num_classes, vmax=1)
    
    axes[1].set_title('Probabilistic Boundary')
    axes[1].contourf(xx, yy, P * C, cmap=cmap, alpha=0.5)
    plt.show()


def plot_data(ax, X, Y):
    num_classes = len(np.unique(Y))
    cmap = plt.cm.jet
    pmap = plt.cm.cividis_r
    norm = mpl.colors.Normalize(vmin=0, vmax=num_classes - 1)
        
    # PREDICT
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = .02  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    xy = np.c_[xx.ravel(), yy.ravel()]
    

    # PLOTS
    ax.scatter(X[:, 0], X[:, 1], c=Y, cmap=plt.cm.jet)

    ax.set_xlim(xx.min()+h, xx.max()-h)
    ax.set_ylim(yy.min()+h, yy.max()-h)
    ax.set_xticks(())
    ax.set_yticks(())
    ax.set_aspect('equal')

    ax.set_title('Data Distribution')
    # Plot also the training point
    

## Comparison of classification models: Accuracy and efficiency

Recall that the goal of classification is to find a function that separates the data into positive/negative labels. In the case of a linear classifier, this reduces to finding a set of parameters $w^\star$ such that, $$ \begin{align} w^\star &= \arg \min_w \sum_{i=1}^{N} \left[y_i\neq \text{sign} (w^\top x_i) \right] \\ &= \arg \min_w \sum_{i=1}^{N} l_{0/1} (w; x_i, y_i) \end{align}.$$ 

The problem with the $l_{0/1}$ loss, is that it is non-convex (and non-differentiable), hence in practice, it is more convenient to use other more efficient surrogate losses. 

We have introduced a few classification models in Module 1, including *logistic regression*, *Naive Bayes model*, and *neural networks*. These models are designed to address different challanges in supervised learning.  In the following, you will take a closer look at the behavior of these models, and try to answer the following questions:

### Accuracy 
1. Fit each model to different datasets. Adjust the noise level, and see how each model's behavior changes in response to the noise. 
2. Which model is the most robust? 
3. Which model achieves the best accuracy? 




In [17]:
rcParams['figure.figsize'] = (20, 15)
rcParams['font.size'] = 16

# %% FLOAT WIDGETS
noise_widget = ipywidgets.FloatSlider(
    value=0.2, min=0, max=1, step=0.01, description='Noise:', continuous_update=False)

def evaluate_classfier(dataset, model, noise):
    np.random.seed(0)
    X, Y = get_classification_dataset(dataset, noise=noise)
    num_classes = len(np.unique(Y))
    X = X[:, :2]
    if model == 'Naive Bayes':
        model = GaussianNB().fit(X, Y)
        mean, covariance = model.theta_, [np.diag(model.sigma_[i]) for i in range(len(model.classes_))]
    elif model == 'LogisticRegression':
        model = LogisticRegression().fit(X, Y)
        mean, covariance = None, None
    elif model == 'Neural Net':
        model = MLPClassifier().fit(X, Y)
        mean, covariance = None, None

    plot_model(model, X, Y, mean, covariance)
    
interact(evaluate_classfier, 
         dataset=['2-blobs', 'linear', '3-blobs', '4-blobs', 'circles', 'moons', 'wine'], 
         model=['Naive Bayes','LogisticRegression', 'Neural Net'], noise=noise_widget);


interactive(children=(Dropdown(description='dataset', options=('2-blobs', 'linear', '3-blobs', '4-blobs', 'cir…

## Comparison of time complexity and label complexity: random sampling

While there may not be a clear winner under all evaluation criteria, from the above demo, we can tell that certain models have their edges if one only focuses on a single (or a subset of) performance measure(s). 

In addition to the **accuracy** and **time efficiency**, another important evaluation criterion for assessing a supervised learning model is *how many training examples* are needed to achieve a target accuracy.  This is often known as the **label complexity**, or **data efficiency** of a supervised model.

In the next task, you will examine how each model behaves as a function of the amount of training examples available. As you go through the code, try to answer the following questions:

1. Which model is the most costly to train in terms of the time complexity (i.e. wall-clock time)?
2. Does one model behavior strictly better than another? If yes, under what criteria? 


In [32]:
import time

def split(X, y, train_size, test_size): 
    
    X_train, X_pool, y_train, y_pool = train_test_split( 
        X, y, train_size = train_size, random_state=42) 
    
    unlabeled_instances, X_test, labels, y_test = train_test_split( 
        X_pool, y_pool, test_size = test_size, random_state=42) 

    return X_train, y_train, X_test, y_test, unlabeled_instances, labels 


def compare_label_complexity(dataset, noise):
    np.random.seed(0)
    X, Y = get_classification_dataset(dataset, noise=noise)

    num_samples = X.shape[0]
    num_trials = 1

    incremental_train_size = int(num_samples * .1)
    init_train_ratio = .05
    test_ratio = 0.20

    num_batches = int(num_samples*(1-test_ratio-init_train_ratio)/incremental_train_size)

    X_INIT, Y_INIT, X_TEST, Y_TEST, X_UNLABELED, Y_UNLABELED = split( 
            X,Y, init_train_ratio, test_ratio) 
    
    num_classes = len(np.unique(Y))

    # initialize performance measure vectors: accuracy and time complexity
    ac = np.zeros((num_trials, num_batches+2, len(classifiers)))
    tc = np.zeros((num_trials, num_batches+2, len(classifiers)))

    # we consider to iteratively increase the number of training examples, with increment ne_incremental
    ne_incremental = [incremental_train_size + i * incremental_train_size for i in range(num_batches+1)]
    ne = [X_INIT.shape[0]] + ne_incremental

    # projected = pca.fit_transform(digits.data)

    # run each models n_trials times and take the average of their accuracy 
    for i in range(num_trials): 
        print('trial', i+1, '/', num_trials)
        
        X_train = X_INIT.copy()
        y_train = Y_INIT.copy()
        X_test = X_TEST.copy()
        y_test = Y_TEST.copy()
        X_pool = X_UNLABELED.copy()
        y_pool = Y_UNLABELED.copy()

        for j in range(num_batches+2):
            # randomly sampling incremental_train_size points
            if j == 0:
                increment = 0
            if j == 1:
                increment = incremental_train_size-X_train.shape[0]
            if j > 1:
                increment = incremental_train_size

            query_idx = np.random.choice(range(len(X_pool)), size=increment, replace=False)

            X_new, y_new = X_pool[query_idx], y_pool[query_idx]

            X_train = np.concatenate((X_train, X_new))
            y_train = np.concatenate((y_train, y_new))    
        
            # update the remaining unlabeled pool
            X_pool, y_pool = np.delete(X_pool, query_idx, axis=0), np.delete(y_pool, query_idx, axis=0)
            
            # train model 
            # iterate over classifiers
            k=0
            for name, clf in zip(names, classifiers):
                start = time.process_time()
                model = clf.fit(X_train, y_train) 
                mean, covariance = None, None
                #print(y_pool.shape[0])
                
                traintime = time.process_time() - start
                score = clf.score(X_test, y_test)
                
                ac[i][j][k] = score
                tc[i][j][k] = traintime
                k +=1

    rcParams['figure.figsize'] = (15, 5)
    rcParams['font.size'] = 16

    # fig = plt.figure()
    # ax=fig.add_subplot(111)

    ax_data = plt.subplot(1, 3, 1)
    plot_data(ax_data, X,Y)

    #accuracy vs training size
    ax_ac = plt.subplot(1, 3, 2)
    ac_lines = ax_ac.plot(ne, np.mean(ac, axis=0).tolist())
    ax_ac.legend(ac_lines, names)
    ax_ac.set_ylabel('accuracy')
    ax_ac.set_title('accuracy vs training size')

    #time complexity vs training size
    ax_tc = plt.subplot(1, 3, 3)
    tc_lines = ax_tc.plot(ne, np.mean(tc, axis=0).tolist())
    ax_tc.legend(tc_lines, names)
    ax_tc.set_ylabel('time (s)')
    ax_tc.set_title('time vs training size')

    plt.tight_layout()
    plt.show()


# specify classifiers
names = [
         'LogisticRegression',
         'Naive Bayes',
         'Neural Net'
        ]

classifiers = [ 
    LogisticRegression(),
    GaussianNB(),
    MLPClassifier(alpha=1, max_iter=1000)
]

interact(compare_label_complexity, 
         dataset=['2-blobs', 'linear', '3-blobs', '4-blobs', 'circles', 'moons', 'wine'], 
         noise=noise_widget);


interactive(children=(Dropdown(description='dataset', options=('2-blobs', 'linear', '3-blobs', '4-blobs', 'cir…

## Outlook: cost-effective data acquisition: Uncertainty sampling for logistic regression

In the previous task, you have seen how the model's performance evolves a it collects more labeled data. An immediate followup question is

* What data should one collect, so that one can most effectively improve the performance of a model?
* Can we do better than randomly collecting more labels?

This is indeed a critical question known as **Active Learning**. In fact, a lot of real-world machine learning applications fit into this category. 

Can you think of any scenarios in the cybersecurity domain, where the label complexity is an important concern?


In [17]:
rcParams['figure.figsize'] = (16, 5)
rcParams['font.size'] = 16

queried_set = {}
def uncertainty_sampling(dataset, criterion, noise):    
    query_button = ipywidgets.Button(description="Query new point")
    update_button = ipywidgets.Button(description="Update Model")
    restart_button = ipywidgets.Button(description="Restart")
    X, Y = get_classification_dataset(dataset, 200, noise=noise)
    num_classes = len(np.unique(Y)) - 1
    X = X[:, :2]

    indexes = np.arange(X.shape[0])
    index_set = set([i for i in indexes])


    def plot(model, X, Y, queried_set, next_idx=None, display_query=True):
        neg_i = np.where(Y == 0)[0]
        pos_i = np.where(Y == 1)[0]

        queried_idx = [i for i in queried_set]
        non_queried_idx = [i for i in index_set.difference(queried_set)]

        qX, qY = X[queried_idx], Y[queried_idx]
        nX, nY = X[non_queried_idx], Y[non_queried_idx]

        # Model prediction contours.
        x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
        y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
        h = .02 
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
        xy = np.c_[xx.ravel(), yy.ravel()]
        P = model.predict_proba(xy).max(axis=1).reshape(xx.shape)
        C = model.predict(xy).reshape(xx.shape)
        H = -(model.predict_proba(xy) * model.predict_log_proba(xy)).sum(axis=1).reshape(xx.shape)
        
        # PLOTS
        fig, axes = plt.subplots(1, 2)
        axes[0].set_title('Classification Boundary')
        axes[0].contourf(xx, yy, C, cmap=plt.cm.jet, alpha=0.5, vmin=0, vmax=num_classes)
        
        if criterion == 'max-entropy':
            axes[1].set_title('Entropy')
            cf = axes[1].contourf(xx, yy, H, cmap=plt.cm.cividis_r, alpha=0.5)
            m = plt.cm.ScalarMappable(cmap=plt.cm.cividis_r)
            m.set_array(H)
            cbar = plt.colorbar(m, ax=axes[1])  
            cbar.set_label('Predicted Entropy', rotation=270, labelpad=20)
            
        elif criterion == 'min-probability':
            axes[1].set_title('Probability')
            cf = axes[1].contourf(xx, yy, P, cmap=plt.cm.cividis_r, alpha=0.5)
            m = plt.cm.ScalarMappable(cmap=plt.cm.cividis_r)
            m.set_array(P)
            cbar = plt.colorbar(m, ax=axes[1])  
            cbar.set_label('Predicted Probability', rotation=270, labelpad=20)

        # Plot also the training points
        for ax in axes:
            ax.scatter(qX[:, 0], qX[:, 1], c=qY, marker='o', s=200, cmap=plt.cm.jet, vmin=0, vmax=num_classes)
            ax.scatter(nX[:, 0], nX[:, 1], c=nY, marker='o', alpha=0.3, s=20, cmap=plt.cm.jet, vmin=0, vmax=num_classes)
            
            if next_idx is not None:
                ax.scatter(X[[next_idx], 0], X[[next_idx], 1], c=Y[[next_idx]], s=400, marker='*',
                           cmap=plt.cm.jet, vmin=0, vmax=num_classes)
            
            ax.set_xlim(xx.min(), xx.max())
            ax.set_ylim(yy.min(), yy.max())
            ax.set_xticks(())
            ax.set_yticks(())

        IPython.display.clear_output(wait=True)
        IPython.display.display(plt.gcf())
        plt.close()

        if display_query:
            display(query_button)
        else:
            display(update_button)
        display(restart_button)

    def update_model(b):
        global queried_set, model

        queried_idx = [i for i in queried_set]
        model = LogisticRegression(C=10).fit(X[queried_idx], Y[queried_idx])

        plot(model, X, Y, queried_set, next_idx=None, display_query=True)
    
    def restart(b):
        global queried_set
        queried_set = set()
        classes = np.unique(Y)
        for c in classes:
            i = np.random.choice(np.where(Y == c)[0])
            queried_set.add(i)
        update_model(None)

    def append_point(b):
        global queried_set, model
        
        query_points = X
        probs = model.predict_proba(X).max(axis=1)
        H = model.predict_log_proba(X) * model.predict_proba(X)
        H = H.sum(axis=1)

        queried_idx = [i for i in queried_set]
        probs[queried_idx] = float('Inf')
        H[queried_idx] = float('Inf')

        if criterion == 'max-entropy':
            i = np.argmin(H)
        elif criterion == 'min-probability':
            i = np.argmin(probs)

        plot(model, X, Y, queried_set,  i, display_query=False)
        queried_set.add(i)

    query_button.on_click(append_point)
    update_button.on_click(update_model)
    restart_button.on_click(restart)

    restart(None);
    
interact(uncertainty_sampling, 
         dataset=['linear', 'imbalanced', '2-blobs', '3-blobs', '4-blobs', 'iris', 'circles', 'moons'], 
         criterion=['min-probability', 'max-entropy'],
         noise=ipywidgets.FloatSlider(value=0.25, min=0, max=1, step=0.01, readout_format='.2f',
                                      continuous_update=False));

interactive(children=(Dropdown(description='dataset', options=('linear', 'imbalanced', '2-blobs', '3-blobs', '…

## Exercise: Spam filtering

In the following, try out the above supervised models on the following spam classification task. For your convenience, we have provided the data loader. Try to plug in the logistic regression, Naive Bayes, and multi-layer perceptron models to this dataset.

The dataset corresponds to a collection of spam e-mails and non-spam e-mails from postmasters and individuals who had filed spam, or who had filed work and personal e-mails collected. The authors (Mark Hopkins, Erik Reeber, George Forman, Jaap Suermondt) provided a detailed description of the spambase dataset [here](https://archive.ics.uci.edu/ml/datasets/spambase). 

Note that the dimensionality of this dataset > 3, hence we cannot directly visualize the model behavior as what we did above. If time permits, answer the following questions:
* Is there any dimensionality reduction technique you can recall from this course to help visualizing the dataset?
* Report the classification accuracy on the test data (use train/test split of 80:20).

In [ ]:
import csv
import pandas as pd
from sklearn.decomposition import PCA

def load_data(Train=False):

    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/spambase/spambase.data'
    df = pd.read_csv(url, header=None)

    # retrieve the array
    data = df.values
    # split into input and output elements
    X, y = data[:, :-1], data[:, -1]
    del data

    pca = PCA(n_components=2)
    # X = pca.fit_transform(X)
    X = StandardScaler().fit_transform(X)
    # X_test_projected = pca.transform(X_test)

    if Train:
        # returns X_train, X_test, y_train, y_test
        return train_test_split(X, y, test_size=0.2, random_state=RandomState())
    else:
        return X, y